In [15]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from collections import Counter
import string
import warnings
warnings.filterwarnings('ignore')
import warnings
warnings.filterwarnings('ignore')

In [16]:
df = pd.read_csv(r"Final Dataset.csv")
df.head()

,subject,message,label
0,job posting - apple-iss research center,content - length : 3386 apple-iss research cen...,0
1,query : letter frequencies for text identifica...,i am posting this inquiry for sergei atamas ( ...,0
2,risk,a colleague and i are researching the differin...,0
3,request book information,earlier this morning i was on the phone with a...,0
4,call for abstracts : optimality in syntactic t...,content - length : 4437 call for papers is the...,0


In [17]:
df.isnull().sum()

subject    0
message    0
label      0
dtype: int64

In [18]:
df['sub_mssg']=df['subject']+df['message']
df.head()

,subject,message,label,sub_mssg
0,job posting - apple-iss research center,content - length : 3386 apple-iss research cen...,0,job posting - apple-iss research centercontent...
1,query : letter frequencies for text identifica...,i am posting this inquiry for sergei atamas ( ...,0,query : letter frequencies for text identifica...
2,risk,a colleague and i are researching the differin...,0,riska colleague and i are researching the diff...
3,request book information,earlier this morning i was on the phone with a...,0,request book informationearlier this morning i...
4,call for abstracts : optimality in syntactic t...,content - length : 4437 call for papers is the...,0,call for abstracts : optimality in syntactic t...


In [19]:
df.drop('subject',axis=1,inplace=True)

In [20]:
df.head()

,message,label,sub_mssg
0,content - length : 3386 apple-iss research cen...,0,job posting - apple-iss research centercontent...
1,i am posting this inquiry for sergei atamas ( ...,0,query : letter frequencies for text identifica...
2,a colleague and i are researching the differin...,0,riska colleague and i are researching the diff...
3,earlier this morning i was on the phone with a...,0,request book informationearlier this morning i...
4,content - length : 4437 call for papers is the...,0,call for abstracts : optimality in syntactic t...


In [21]:
df.drop('message',axis=1,inplace=True)

In [22]:
df.head()

,label,sub_mssg
0,0,job posting - apple-iss research centercontent...
1,0,query : letter frequencies for text identifica...
2,0,riska colleague and i are researching the diff...
3,0,request book informationearlier this morning i...
4,0,call for abstracts : optimality in syntactic t...


In [23]:
import re

In [24]:
#REPLACING NUMBERS
df['sub_mssg']=df['sub_mssg'].str.replace(r'\d+(\.\d+)?', 'numbers')
df['sub_mssg'][5]

'm . a . in scandinavian linguisticsm . a . in scandinavian linguistics at the university of tromsoe numbers-numbers ( corrected version ) the linguistics section of the institute for language and literature at the university of tromsoe  norway  offers a one year course leading up to a master of arts degree  starting the fall of numbers . the course is designed to prepare students for research in generative syntax and phonology with special attention to the scandinavian languages . the course is open to students from all countries with a background in linguistics corresponding roughly to a b . a . with a major in linguistics  or a norwegian cand . mag . the course is equivalent to numbers e . c . t . s credits . instruction will be in english . proficiency in a scandinavian language is not required  nor is previous knowledge of scandinavian grammar . those who wish may follow a course in norwegian for foreigners during the whole year . students pay no tuition  only a nominal registrati

In [25]:
#CONVRTING EVERYTHING TO LOWERCASE
df['sub_mssg']=df['sub_mssg'].str.lower()
#REPLACING NEXT LINES BY 'WHITE SPACE'
df['sub_mssg']=df['sub_mssg'].str.replace(r'\n'," ") 
# REPLACING EMAIL IDs BY 'MAILID'
df['sub_mssg']=df['sub_mssg'].str.replace(r'^.+@[^\.].*\.[a-z]{2,}$','MailID')
# REPLACING URLs  BY 'Links'
df['sub_mssg']=df['sub_mssg'].str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','Links')
# REPLACING CURRENCY SIGNS BY 'MONEY'
df['sub_mssg']=df['sub_mssg'].str.replace(r'£|\$', 'Money')
# REPLACING LARGE WHITE SPACE BY SINGLE WHITE SPACE
df['sub_mssg']=df['sub_mssg'].str.replace(r'\s+', ' ')

# REPLACING LEADING AND TRAILING WHITE SPACE BY SINGLE WHITE SPACE
df['sub_mssg']=df['sub_mssg'].str.replace(r'^\s+|\s+?$', '')
#REPLACING CONTACT NUMBERS
df['sub_mssg']=df['sub_mssg'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$','contact number')
#REPLACING SPECIAL CHARACTERS  BY WHITE SPACE 
df['sub_mssg']=df['sub_mssg'].str.replace(r"[^a-zA-Z0-9]+", " ")

In [28]:
len(df['sub_mssg'][7])

1203

In [14]:
# import spacy
# nlp = spacy.load("en_core_websm")

# doc = nlp("Mando talked for 3 hours although talking isn't his thing")
# doc = nlp("eating eats eat ate adjustable rafting ability meeting better")
# for token in doc:
#     print(token, " | ", token.lemma)

ModuleNotFoundError: No module named 'spacy'

In [20]:
df['sub_mssg'][0]

'job posting apple iss research centercontent length numbers apple iss research center a us Money numbers million joint venture between apple computer inc and the institute of systems science of the national university of singapore located in singapore is looking for a senior speech scientist the successful candidate will have research expertise in computational linguistics including natural language processing and english and chinese statistical language modeling knowledge of state of the art corpus based n gram language models cache language models and part of speech language models are required a text to speech project leader the successful candidate will have research expertise expertise in two or more of the following areas computational linguistics including natural language parsing lexical database design and statistical language modeling text tokenization and normalization prosodic analysis substantial knowledge of the phonology syntax and semantics of chinese is required knowl

In [30]:
import nltk
words = set(nltk.corpus.words.words())

sent = "Io andiamo to the beach with my amico."
" ".join(w for w in nltk.wordpunct_tokenize(sent) if w.lower() in words or not w.isalpha())

LookupError: 
**********************************************************************
  Resource [93mwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('words')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/words[0m

  Searched in:
    - 'C:\\Users\\Prakhar Jadaun/nltk_data'
    - 'c:\\Users\\Prakhar Jadaun\\AppData\\Local\\Programs\\Python\\Python39\\nltk_data'
    - 'c:\\Users\\Prakhar Jadaun\\AppData\\Local\\Programs\\Python\\Python39\\share\\nltk_data'
    - 'c:\\Users\\Prakhar Jadaun\\AppData\\Local\\Programs\\Python\\Python39\\lib\\nltk_data'
    - 'C:\\Users\\Prakhar Jadaun\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
